In [3]:
!pip install sentence-transformers faiss-cpu chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 69.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 70.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 78.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install langchain langchain-community pymupdf docx2txt

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 80.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 71.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 1.2.19
    Uninstalling SQLAlchemy-1.2.19:
      Successfully uninstalled SQLAlchemy-1.2.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xmanager 0.7.1 requires sqlalchemy==1.2.19, but you have sqlalchemy 2.0.45 which is incompatible.


In [5]:
from langchain_community.document_loaders import DirectoryLoader

In [6]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

In [7]:
dir_loader = DirectoryLoader(
    "/kaggle/input",
    glob="**/*.pdf",
    loader_cls=PyMuPDFLoader,
    show_progress=False
)

In [8]:
chunks = dir_loader.load()

In [9]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from sklearn.metrics.pairwise import cosine_similarity
from typing import List,Any, Dict
import os

2025-12-27 11:35:45.435846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766835345.727538      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766835345.807960      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766835346.542514      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766835346.542570      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766835346.542573      55 computation_placer.cc:177] computation placer alr

In [10]:
class EmbeddingManager:

    def __init__(self,model_name='all-MiniLM-L6-v2'):
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        try:
            print(f"Embedding Model Name: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print("Model Loaded Succesfully")
            print(f"Embedding Dim: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print("Error in loading model")
            raise

    def generate_embeddings(self, texts:List[str]) -> np.ndarray:
        if not self.model:
            raise ValueError("Model Not Loaded")

        print(f"Generating embeddings for texts of len: {len(texts)}")
        embeddings = self.model.encode(texts,show_progress_bar=True)
        print(f"Generated embeddings of shape: {embeddings.shape}")
        return embeddings




    

In [11]:
embedding_manager = EmbeddingManager()

Embedding Model Name: all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model Loaded Succesfully
Embedding Dim: 384


### Vector Store

In [12]:
class VectorStore:

    def __init__(self,collection_name:str="pdf_docs",persist_directory:str="/kaggle/working/"):
        self.collection_name=collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection=None
        self._initialise_store()

    def _initialise_store(self):

        try:
            #create persist chromadb client
            os.makedirs(self.persist_directory,exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
    
            #get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"Description":"PDF Doc emmbedings for RAG"}
            )
    
            print(f"VectorStore is created with name: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error at initialising vector store {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings=np.ndarray):
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents doesn't match embeddings")

        print(f"Adding {len(documents)} to vector store")

        #Data for chromadb
        ids = []
        metadatas = []
        document_text = []
        embedding_list = []

        for i,(doc,emb) in enumerate(zip(documents,embeddings)):

            #generating ids
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
        
            #preparing metadata
            metadata = dict()
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            document_text.append(doc.page_content)

            embedding_list.append(emb.tolist())

        try:
            self.collection.add(
                ids = ids,
                embeddings = embedding_list,
                metadatas = metadatas,
                documents = document_text
            )

            print(f"successfully added { len(documents)} documents to vector store") 
            print(f"Number of documents in collection {self.collection.count()}")
        except Exception as e:
            print(f"Error in adding documents to collection {e}")
            raise

In [13]:
vector_store = VectorStore()
vector_store

VectorStore is created with name: pdf_docs
Existing documents in collection: 0


In [14]:
texts = [doc.page_content for doc in chunks]

embeddings = embedding_manager.generate_embeddings(texts)

vector_store.add_documents(chunks,embeddings)

Generating embeddings for texts of len: 1098


Batches:   0%|          | 0/35 [00:00<?, ?it/s]

Generated embeddings of shape: (1098, 384)
Adding 1098 to vector store
successfully added 1098 documents to vector store
Number of documents in collection 1098


In [15]:
class RAGRetriever:

    def __init__(self,vector_store:VectorStore, embedding_manager:EmbeddingManager):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query:str, top_k : int=5, score_threshold:float=0.0) -> List[Dict[str,Any]]:
        print("retrieving documents for query: ", query)

        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        try:
            res = self.vector_store.collection.query(
                query_embeddings = [query_embedding.tolist()],
                n_results = top_k
            )

            retrieved_docs = []

            if res['documents'] and res['documents'][0]:
                documents = res['documents'][0]
                metadatas = res['metadatas'][0]
                distances = res['distances'][0]
                ids = res['ids'][0]

                for i,(document,metadata,distance,doc_id) in enumerate(zip(documents,metadatas,distances,ids)):
                    similarity_score = 1-distance
    
                    if similarity_score>score_threshold:
                        retrieved_docs.append({
                            'id':doc_id,
                            'content':document,
                            'metadata':metadata,
                            'similarity_score':similarity_score,
                            'distance':distance,
                            'rank':i+1
                        })
    
                print(f"Retrieved {len(retrieved_docs)} document after filtering ")
            else:
                print("No documents retrieved")

            return retrieved_docs
        except Exception as e:
            print("Error while retrieving")
            raise
            
            

In [16]:
rag_retriever = RAGRetriever(vector_store,embedding_manager)
rag_retriever

In [17]:
rag_retriever.retrieve("What is probability")

retrieving documents for query:  What is probability
Generating embeddings for texts of len: 1


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings of shape: (1, 384)
Retrieved 1 document after filtering 


[{'id': 'doc_920d0237_57',
  'content': '2\nProbability\n2.1\nIntroduction\nProbability theory is nothing but common sense reduced to calculation. — Pierre Laplace,\n1812\nIn the previous chapter, we saw how probability can play a useful role in machine learning. In\nthis chapter, we discuss probability theory in more detail. We do not have to space to go into\ngreat detail — for that, you are better off consulting some of the excellent textbooks available\non this topic, such as (Jaynes 2003; Bertsekas and Tsitsiklis 2008; Wasserman 2004). But we will\nbrieﬂy review many of the key ideas you will need in later chapters.\nBefore we start with the more technical material, let us pause and ask: what is probability?\nWe are all familiar with the phrase “the probability that a coin will land heads is 0.5”. But what\ndoes this mean? There are actually at least two different interpretations of probability. One is\ncalled the frequentist interpretation. In this view, probabilities represent l

## Integration with LLM 

In [18]:
!pip install -U --no-cache-dir \
  langchain==0.2.16 \
  langchain-core==0.2.38 \
  langchain-openai==0.1.23
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 274.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 156.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 263.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 256.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 256.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.4.37
    Uninstalling langsmith-0.4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

In [20]:
model_path = "/kaggle/input/gemma/transformers/2b-it/3"

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

In [23]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.2,
    do_sample=True,
    return_full_text=False 
)

llm = HuggingFacePipeline(pipeline=pipe)


Device set to use cpu
/tmp/ipykernel_55/1595155424.py:11: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [24]:
def rag_simple(query,retriever, llm, top_k=3):
    res = retriever.retrieve(query,top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in res]) if res else ""

    if not context:
        return "No relevant context"

    prompt = f"""Use the following context to answer the question consisely
            Context:
            {context}
            Question:
            {query}
            
            Answer:"""

    response = llm.invoke([prompt])
    return response

In [25]:
answer = rag_simple(
    query="what is Maximum entropy derivation of the Gaussian",
    retriever=rag_retriever,
    llm=llm
)

retrieving documents for query:  what is Maximum entropy derivation of the Gaussian
Generating embeddings for texts of len: 1


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings of shape: (1, 384)
Retrieved 3 document after filtering 


In [26]:
answer

'\n            The Maximum entropy derivation of the Gaussian is given by\n            h(N(μ, Σ)) = 1\n2 ln\n\x14\n(2πe)D|Σ|'

In [27]:
def rag_advanced(query,retriever, llm, top_k=3,min_score=0.2,return_context=False):
    res = retriever.retrieve(query,top_k=top_k)
    if not res:
        return {'answer':"No relevant context","sources":[],"confidence_score":0.0,"context":''}

    context = "\n\n".join([doc['content'] for doc in res]) if res else ""

    sources = [{
        "source":doc['metadata'].get('source_file',doc['metadata'].get('source','unknown')),
        'page':doc['metadata'].get('page','unknown'),
        'score':doc['similarity_score'],
        'preview':doc['content'][:300]+"...",
    } for doc in res]

    confidence = max(doc['similarity_score'] for doc in res)
    prompt = f"""Use the following context to answer the question consisely: \nContext: \n {context}
            Question: \n{query}
            Answer:"""

    response = llm.invoke([prompt])

    output = {
        'answer':response,
        'sources':sources,
        'confidence':confidence
    }
    return output

In [ ]:
answer = rag_advanced(
    query="explain the use of linear regression",
    retriever=rag_retriever,
    llm=llm
)

In [ ]:
answer['answer']

In [ ]:
answer['sources']

In [ ]:
!pip install typesense

In [ ]:
import typesense

In [33]:
client = typesense.Client({
    'nodes': [{
        'host': 'kpuysm2df54h67x9p-1.a1.typesense.net', 
        'port': '443',
        'protocol': 'https'
        }],
    'api_key': 'yoyaOpBaoCX65ZgeryXnCOZRJNAqCjBZ',
    'connection_timeout_seconds': 2
})

In [31]:
books_schema = {
    'name': 'books',
    'fields': [
        {'name': 'title', 'type': 'string'},
        {'name': 'authors', 'type': 'string[]', 'facet': True},
        {'name': 'publication_year', 'type': 'int32', 'facet': True},
        {'name': 'ratings_count', 'type': 'int32'},
        {'name': 'average_rating', 'type': 'float'}
     ],
    'default_sorting_field': 'ratings_count'
}

In [ ]:
client.collections.create(books_schema)

In [36]:
with open("/kaggle/input/books-data/books.jsonl",'r',encoding='utf-8') as f:
    data = f.read()
    client.collections['books'].documents.import_(data)

In [41]:
search_parameters={
    'q':"harry potter",
    'query_by': "title, authors",
    "filter_by":"publication_year:<1998",
    "sort_by": "ratings_count:desc",
}
client.collections['books'].documents.search(search_parameters)

{'facet_counts': [],
 'found': 1,
 'hits': [{'document': {'authors': ['J.K. Rowling', ' Mary GrandPré'],
    'average_rating': 4.44,
    'id': '2',
    'image_url': 'https://images.gr-assets.com/books/1474154022m/3.jpg',
    'publication_year': 1997,
    'ratings_count': 4602479,
    'title': "Harry Potter and the Philosopher's Stone"},
   'highlight': {'title': {'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}],
   'text_match': 1157451471441100921,
   'text_match_info': {'best_field_score': '2211897868288',
    'best_field_weight': 15,
    'fields_matched': 1,
    'num_tokens_dropped': 0,
    'score': '1157451471441100921',
    'tokens_matched': 2,
    'typo_prefix_score': 0}}],
 'out_of': 9979,
 'page': 1,
 'request_params': {'collection_name